# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    for state in graph.nodes():
        if graph.node[state]['demand'] < 0:
            graph.add_edge('S', state)
            graph.edge['S'][state]['capacity'] = -graph.node[state]['demand']
        elif graph.node[state]['demand'] > 0:
            graph.add_edge(state, 'T')
            graph.edge[state]['T']['capacity'] = graph.node[state]['demand']

    dic = {}
    for (s1, s2) in graph.edges():
        dic[(s1, s2)] = graph.edge[s1][s2]['capacity']
    nx.set_edge_attributes(graph, 'capacity', dic)
    flow_value = nx.maximum_flow_value(graph, s = "S", t = "T")
    flow = nx.maximum_flow(graph, s = "S", t = "T")[1]
    graph.remove_node('S')
    graph.remove_node('T')
    return flow

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    dic = {}
    for source in flow:
        for sink in flow[source]:
            if source == 'S' or sink == 'T':
                continue
            if source in dic:
                if sink in dic:
                    dic[source], dic[sink] = dic[source] - flow[source][sink], dic[sink] + flow[source][sink]
                elif sink not in dic:
                    dic[source], dic[sink] = dic[source] - flow[source][sink], flow[source][sink]
            elif source not in dic:
                if sink in dic:
                    dic[source], dic[sink] = -flow[source][sink], dic[sink] + flow[source][sink]
                elif sink not in dic:
                    dic[source], dic[sink] = -flow[source][sink], flow[source][sink]
    return dic
    

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
flow
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True


In [7]:
flow

{'AL': {'FL': 1, 'GA': 0, 'MS': 5, 'T': 1, 'TN': 0},
 'AR': {'LA': 0, 'MO': 8, 'MS': 0, 'OK': 0, 'T': 1, 'TN': 0, 'TX': 0},
 'AZ': {'CA': 0, 'NM': 0, 'NV': 0, 'T': 1, 'UT': 16},
 'CA': {'AZ': 16, 'NV': 16, 'OR': 16},
 'CO': {'KS': 0, 'NE': 0, 'NM': 0, 'OK': 15, 'T': 1, 'UT': 0, 'WY': 0},
 'CT': {'MA': 0, 'NY': 0, 'RI': 1, 'T': 1},
 'DC': {'MD': 0, 'T': 1, 'VA': 0},
 'DE': {'MD': 0, 'NJ': 6, 'PA': 0, 'T': 1},
 'FL': {'AL': 0, 'GA': 0, 'T': 1},
 'GA': {'AL': 7, 'FL': 0, 'NC': 0, 'SC': 0, 'T': 1, 'TN': 2},
 'IA': {'IL': 0, 'MN': 4, 'MO': 0, 'NE': 0, 'SD': 0, 'T': 1, 'WI': 0},
 'ID': {'MT': 3, 'NV': 0, 'OR': 0, 'T': 1, 'UT': 3, 'WA': 15, 'WY': 8},
 'IL': {'IA': 0, 'IN': 3, 'KY': 0, 'MO': 0, 'T': 1, 'WI': 0},
 'IN': {'IL': 0, 'KY': 2, 'MI': 0, 'OH': 0, 'T': 1},
 'KS': {'CO': 0, 'MO': 0, 'NE': 6, 'OK': 0, 'T': 1},
 'KY': {'IL': 4, 'IN': 0, 'MO': 0, 'OH': 2, 'T': 1, 'TN': 0, 'VA': 2, 'WV': 0},
 'LA': {'AR': 9, 'MS': 1, 'T': 1, 'TX': 0},
 'MA': {'CT': 0, 'NH': 3, 'NY': 0, 'RI': 0, 'T': 1, 'VT'